## 📦 Setup

In [1]:
# 1. Carregar dados + 
import sys
sys.path.append('..')  # Adiciona o diretório pai
from src.utils.io.io_local import *
from src.utils.io.io_clearml import *

from src.utils.io import load_dataframe
from config import config_custom as config
from src.pipelines.pipeline_processamento import executar_pipeline_processamento
from src.pipelines.pipeline_features import executar_pipeline_features
from src.pipelines import treinar_pipeline_completo, treinar_rapido

df_raw = load_dataframe('../dados/2025.05.14_thermal_confort_santa_maria_brazil_.csv')
print(f'Dados brutos: {df_raw.shape}')

Dados brutos: (1720, 40)


# Pipelines

## Pipeline processamento

In [2]:
# 1. Carregar dados + # 2. PROCESSAMENTO (Limpeza + Imputação)
df_raw = load_dataframe('../dados/2025.05.14_thermal_confort_santa_maria_brazil_.csv')
print(f'Dados brutos: {df_raw.shape}')

# 2. PROCESSAMENTO (Limpeza + Imputação)
df_proc = executar_pipeline_processamento(
    df_raw,
    config_imputacao_customizada=config.CONFIG_IMPUTACAO_CUSTOMIZADA,
    criar_agrupamento_temporal=True,
    nome_coluna_agrupamento='mes-ano'
)
print(f'Após processamento: {df_proc.shape}')
print(f'NAs restantes: {df_proc.isna().sum().sum()}')

Dados brutos: (1720, 40)
🔄 Iniciando pipeline de processamento BASE...
  1️⃣ Aplicando substituições de limpeza...
  2️⃣ Convertendo tipos de dados...
  3️⃣ Imputando valores faltantes...
  4️⃣ Criando agrupamento temporal...
✅ Pipeline BASE concluído! Shape final: (1720, 41)
Após processamento: (1720, 41)
NAs restantes: 0


## Pipeline features

In [3]:
# 3. FEATURES (Codificação + Derivadas + Normalização)
df_feat, artefatos = executar_pipeline_features(
    df_proc,
    # Codificação
    aplicar_codificacao=True,
    metodo_codificacao='label',           # 'label' ou 'onehot'
    sufixo_codificacao='_cod',
    
    # Features derivadas
    criar_features_derivadas=True,
    tipos_features_derivadas=[
        'imc',                            # Índice de Massa Corporal
        'imc_classe',                     # Classe do IMC
        'heat_index',                     # Índice de calor
        'dew_point',                      # Ponto de orvalho
        't*u',                            # Temperatura × Umidade
    ],
    
    # Normalização
    aplicar_normalizacao=True,
    metodo_normalizacao='standard',       # 'standard', 'minmax', 'robust'
    agrupamento_normalizacao='mes-ano',   # Normalizar por grupo
    sufixo_normalizacao='_norm',
)

print(f'Após features: {df_feat.shape}')
print(f'Artefatos criados: {list(artefatos.keys())}')
df_feat.to_csv("../dados/resultados/dados_processados_novas_features.csv")
# Visualizar resultado
df_feat.head()


🎨 Iniciando pipeline de FEATURES...
  1️⃣ Criando features derivadas (5 tipos)...
  2️⃣ Aplicando codificação (label)...
  3️⃣ Aplicando normalização (standard)...
✅ Pipeline FEATURES concluído! Shape final: (1720, 78)
   Novas colunas criadas: 37
Após features: (1720, 78)
Artefatos criados: ['mapeamentos_codificacao', 'colunas_normalizadas']


,data,hora,idade,sexo,peso,altura,vestimenta,p1,p2,p3,...,vel_vento_norm,sd_dirvento_norm,vel_vento_max_norm,chuva_tot_norm,IMC_norm,heat_index_norm,dew_point_norm,t_u_norm,sexo_cod_norm,vestimenta_cod_norm
0,2015-08-05,09:10:00,51,f,77,163,"0,5",0,0,0,...,-1.100945,-1.500344,-1.063319,0.0,1.338207,2.157433,-0.837574,1.048451,-0.942160,-0.453888
1,2015-08-05,09:10:00,50,f,69,170,1,0,0,0,...,-1.100945,-1.500344,-1.063319,0.0,-0.141792,2.157433,-0.837574,1.048451,-0.942160,2.071971
2,2015-08-05,09:10:00,25,m,96,185,1,-1,0,-2,...,-1.100945,-1.500344,-1.063319,0.0,1.068200,2.157433,-0.837574,1.048451,1.061391,2.071971
3,2015-08-05,09:10:00,27,f,98,160,1,0,0,0,...,-1.100945,-1.500344,-1.063319,0.0,4.034041,2.157433,-0.837574,1.048451,-0.942160,2.071971
4,2015-08-05,09:10:00,35,f,56,167,1,1,0,-2,...,-1.100945,-1.500344,-1.063319,0.0,-1.242094,2.157433,-0.837574,1.048451,-0.942160,2.071971


# Pipeline de treinamento

In [ ]:
from config.config_gerais import PARAMS_PADRAO

# Definir coluna alvo e features
coluna_alvo = 'p1'

# Usar apenas features principais para teste
features_treino = [
    'idade', 'sexo_cod', 'peso', 'altura',
    'tmedia', 'ur', 'vel_vento',
]

tipos_modelos ='regressao'

# Filtrar features que existem no DataFrame
features_existentes = [f for f in features_treino if f in df_feat.columns]
print(f"Features para treinamento: {features_existentes}")

# Preparar dados
df_treino = df_feat[features_existentes + [coluna_alvo]].dropna()
print(f"Dataset de treino: {df_treino.shape}")

# 4. TREINAMENTO
print("\n" + "="*60)
print("🤖 INICIANDO PIPELINE DE TREINAMENTO")
print("="*60)

resultado = treinar_pipeline_completo(
    dados=df_treino,
    coluna_alvo=coluna_alvo,
    tipo_problema=tipos_modelos,  # 'classificacao' ou 'regressao'
    params_setup=PARAMS_PADRAO,
    n_modelos_comparar=3,           # Testar top 3 modelos
    otimizar_hiperparametros=True,         # Otimizar hiperparâmetros
    n_iter_otimizacao=10,  # 10 iterações de otimização
    salvar_modelo_final=True,           # Salvar modelo
    nome_modelo="modelo_conforto_termico",
    pasta_modelos="modelos"
)

# Visualizar resultados
print("\n" + "="*60)
print("📊 RESULTADOS DO TREINAMENTO")
print("="*60)

# Nome do melhor modelo
nome_modelo = str(resultado['tabela_comparacao'].index[0])
print(f"\n✓ Melhor modelo: {nome_modelo}")

print(f"\n📈 Métricas principais:")
metricas = resultado['metricas_melhor']
for nome, valor in metricas.items():
    if isinstance(valor, (int, float)):
        print(f"  • {nome}: {valor:.4f}")

print(f"\n💾 Modelo salvo em: {resultado.get('caminho_modelo', 'N/A')}")

print("\n📋 Comparação de modelos (top 5 métricas):")
print(resultado['tabela_comparacao'].head())

2026-02-05 11:13:12,444 - INFO - Comparação de regressão concluída. 3 modelo(s) selecionado(s).
2026-02-05 11:13:12,450 - INFO - ✓ 3 modelo(s) selecionado(s)
2026-02-05 11:13:12,451 - INFO -   Melhor modelo: lr
2026-02-05 11:13:12,451 - INFO -   R2: 0.5642
2026-02-05 11:13:12,452 - INFO - 
ETAPA 3: Otimizando hiperparâmetros (10 iterações)...
2026-02-05 11:13:12,453 - INFO - Otimizando hiperparâmetros do modelo de regressão...
2026-02-05 11:13:19,793 - INFO - Otimização de regressão concluída para LinearRegression
2026-02-05 11:13:19,793 - INFO - ✓ Hiperparâmetros otimizados
2026-02-05 11:13:19,793 - INFO -   R2 após otimização: 0.5662
2026-02-05 11:13:19,793 - INFO - 
ETAPA 4: Finalizando modelo (treinamento em dataset completo)...
2026-02-05 11:13:19,793 - INFO - Finalizando modelo de regressão no dataset completo...
2026-02-05 11:13:20,197 - INFO - Modelo de regressão finalizado com sucesso.
2026-02-05 11:13:20,197 - INFO - ✓ Modelo finalizado
2026-02-05 11:13:20,197 - INFO - 
ETAPA

Transformation Pipeline and Model Successfully Saved

📊 RESULTADOS DO TREINAMENTO

✓ Melhor modelo: lr

📈 Métricas principais:
  • MAE: 0.7393
  • MSE: 0.8368
  • RMSE: 0.9141
  • R2: 0.5642
  • RMSLE: 0.4487
  • MAPE: 0.4440
  • TT (Sec): 0.1560

💾 Modelo salvo em: modelos\modelo_conforto_termico.pkl

📋 Comparação de modelos (top 5 métricas):
                        Model     MAE     MSE    RMSE      R2   RMSLE    MAPE  \
lr          Linear Regression  0.7393  0.8368  0.9141  0.5642  0.4487  0.4440   
ridge        Ridge Regression  0.7393  0.8368  0.9141  0.5642  0.4487  0.4441   
lar    Least Angle Regression  0.7393  0.8368  0.9141  0.5642  0.4487  0.4440   
br             Bayesian Ridge  0.7392  0.8368  0.9141  0.5642  0.4488  0.4443   
huber         Huber Regressor  0.7390  0.8433  0.9178  0.5607  0.4493  0.4513   

       TT (Sec)  
lr        0.156  
ridge     0.146  
lar       0.138  
br        0.142  
huber     0.152  


In [6]:
resultados_10_experimentos = {}
for i in range(10):
    print("\n" + "="*60)
    print(f"🤖 INICIANDO EXPERIMENTO {i+1}/10")
    print("="*60)
    
    resultado = treinar_pipeline_completo(
    dados=df_treino,
    coluna_alvo=coluna_alvo,
    tipo_problema='regressao',  # 'classificacao' ou 'regressao'
    params_setup=PARAMS_PADRAO,
    n_modelos_comparar=3,           # Testar top 3 modelos
    otimizar_hiperparametros=True,         # Otimizar hiperparâmetros
    n_iter_otimizacao=10,  # 10 iterações de otimização
    salvar_modelo_final=True,           # Salvar modelo
    nome_modelo="modelo_conforto_termico",
    pasta_modelos="modelos"
)
    
    resultados_10_experimentos[f'Experimento_{i+1}'] = resultado


2026-02-05 11:14:12,575 - INFO - ============================================================
2026-02-05 11:14:12,576 - INFO - Iniciando pipeline completo de treinamento de REGRESSÃO
2026-02-05 11:14:12,577 - INFO - ============================================================
2026-02-05 11:14:12,577 - INFO - ETAPA 1: Configurando experimento PyCaret (REGRESSÃO)...
2026-02-05 11:14:12,578 - INFO - Criando experimento de REGRESSÃO...
2026-02-05 11:14:12,579 - INFO - Configurando experimento de regressao...



🤖 INICIANDO EXPERIMENTO 1/10
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce GT 1030, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped traini

2026-02-05 11:14:16,743 - INFO - ✓ Experimento de regressao configurado com sucesso
2026-02-05 11:14:16,743 - INFO - ✓ Experimento de regressao configurado
2026-02-05 11:14:16,743 - INFO - 
ETAPA 2: Comparando modelos (selecionando top 3)...
2026-02-05 11:14:16,743 - INFO - Comparando modelos de regressão...


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce GT 1030, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more l

2026-02-05 11:14:42,550 - INFO - Comparação de regressão concluída. 3 modelo(s) selecionado(s).
2026-02-05 11:14:42,555 - INFO - ✓ 3 modelo(s) selecionado(s)
2026-02-05 11:14:42,556 - INFO -   Melhor modelo: lr
2026-02-05 11:14:42,557 - INFO -   R2: 0.5762
2026-02-05 11:14:42,558 - INFO - 
ETAPA 3: Otimizando hiperparâmetros (10 iterações)...
2026-02-05 11:14:42,558 - INFO - Otimizando hiperparâmetros do modelo de regressão...
2026-02-05 11:14:48,050 - INFO - Otimização de regressão concluída para LinearRegression
2026-02-05 11:14:48,051 - INFO - ✓ Hiperparâmetros otimizados
2026-02-05 11:14:48,051 - INFO -   R2 após otimização: 0.5866
2026-02-05 11:14:48,052 - INFO - 
ETAPA 4: Finalizando modelo (treinamento em dataset completo)...
2026-02-05 11:14:48,052 - INFO - Finalizando modelo de regressão no dataset completo...
2026-02-05 11:14:48,420 - INFO - Modelo de regressão finalizado com sucesso.
2026-02-05 11:14:48,420 - INFO - ✓ Modelo finalizado
2026-02-05 11:14:48,420 - INFO - 
ETAPA

...

Transformation Pipeline and Model Successfully Saved

🤖 INICIANDO EXPERIMENTO 10/10
[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce GT 1030, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the 

2026-02-05 11:18:49,784 - INFO - ✓ Experimento de regressao configurado com sucesso
2026-02-05 11:18:49,787 - INFO - ✓ Experimento de regressao configurado
2026-02-05 11:18:49,787 - INFO - 
ETAPA 2: Comparando modelos (selecionando top 3)...
2026-02-05 11:18:49,787 - INFO - Comparando modelos de regressão...


[LightGBM] [Warning] There are no meaningful features which satisfy the provided configuration. Decreasing Dataset parameters min_data_in_bin or min_data_in_leaf and re-constructing Dataset might resolve this warning.
[LightGBM] [Info] This is the GPU trainer!!
[LightGBM] [Info] Total Bins 0
[LightGBM] [Info] Number of data points in the train set: 2, number of used features: 0
[LightGBM] [Info] Using GPU Device: NVIDIA GeForce GT 1030, Vendor: NVIDIA Corporation
[LightGBM] [Info] Compiling OpenCL Kernel with 16 bins...
[LightGBM] [Info] GPU programs have been built
[LightGBM] [Warning] GPU acceleration is disabled because no non-trivial dense features can be found
[LightGBM] [Info] Start training from score 0.500000
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more leaves that meet the split requirements
[LightGBM] [Warning] Stopped training because there are no more l

2026-02-05 11:19:12,370 - INFO - Comparação de regressão concluída. 3 modelo(s) selecionado(s).
2026-02-05 11:19:12,374 - INFO - ✓ 3 modelo(s) selecionado(s)
2026-02-05 11:19:12,375 - INFO -   Melhor modelo: lr
2026-02-05 11:19:12,375 - INFO -   R2: 0.5676
2026-02-05 11:19:12,375 - INFO - 
ETAPA 3: Otimizando hiperparâmetros (10 iterações)...
2026-02-05 11:19:12,377 - INFO - Otimizando hiperparâmetros do modelo de regressão...
2026-02-05 11:19:14,786 - INFO - Otimização de regressão concluída para LinearRegression
2026-02-05 11:19:14,786 - INFO - ✓ Hiperparâmetros otimizados
2026-02-05 11:19:14,787 - INFO -   R2 após otimização: 0.5221
2026-02-05 11:19:14,788 - INFO - 
ETAPA 4: Finalizando modelo (treinamento em dataset completo)...
2026-02-05 11:19:14,788 - INFO - Finalizando modelo de regressão no dataset completo...
2026-02-05 11:19:15,144 - INFO - Modelo de regressão finalizado com sucesso.
2026-02-05 11:19:15,144 - INFO - ✓ Modelo finalizado
2026-02-05 11:19:15,145 - INFO - 
ETAPA

Transformation Pipeline and Model Successfully Saved


In [7]:
tabelas_comparacao = [dicionario["tabela_comparacao"] for dicionario in resultados_10_experimentos.values()] 
tabelas_comparacao =  [tabela.rename(columns={'Model':'Modelos','Accuracy': 'Acurácia', 'AUC': 'AUC', 'Recall': 'Recall', 'Prec.': 'Prec.', 'F1': 'F1'}) for tabela in tabelas_comparacao]
serie_nomes_modelos = tabelas_comparacao[0]['Modelos']
tabelas_comparacao = [tabela.select_dtypes(include='number') for tabela in tabelas_comparacao] 

In [8]:

df_desvio_metricas= pd.concat(tabelas_comparacao).groupby(level=0).std()
df_media_metricas = pd.concat(tabelas_comparacao).groupby(level=0).mean()
df_media_desvio_metricas_str = df_media_metricas.round(2).astype(str) + " ± " + df_desvio_metricas.round(2).astype(str)
df_media_desvio_metricas_str = df_media_desvio_metricas_str.merge(serie_nomes_modelos, left_index=True, right_index=True)
df_media_metricas = df_media_metricas.merge(serie_nomes_modelos, left_index=True, right_index=True)
df_desvio_metricas = df_desvio_metricas.merge(serie_nomes_modelos, left_index=True, right_index=True)
df_desvio_metricas.to_csv('../dados/resultados/desvio_metricas.csv')
df_media_metricas.to_csv('../dados/resultados/media_metricas.csv')
df_media_desvio_metricas_str.to_csv('../dados/resultados/media_e_desvio_metricas_str.csv')

In [11]:
df_media_desvio_metricas_str


,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec),Modelos
ada,0.79 ± 0.02,0.88 ± 0.04,0.94 ± 0.02,0.52 ± 0.02,0.46 ± 0.01,0.43 ± 0.01,0.2 ± 0.02,AdaBoost Regressor
br,0.73 ± 0.01,0.8 ± 0.02,0.9 ± 0.01,0.56 ± 0.01,0.44 ± 0.0,0.43 ± 0.0,0.15 ± 0.01,Bayesian Ridge
dt,0.96 ± 0.03,1.67 ± 0.08,1.29 ± 0.03,0.09 ± 0.05,0.61 ± 0.01,0.64 ± 0.02,0.15 ± 0.02,Decision Tree Regressor
dummy,1.12 ± 0.02,1.84 ± 0.04,1.35 ± 0.01,-0.01 ± 0.0,0.5 ± 0.0,0.65 ± 0.02,0.15 ± 0.02,Dummy Regressor
en,0.96 ± 0.01,1.29 ± 0.02,1.14 ± 0.01,0.29 ± 0.01,0.48 ± 0.0,0.56 ± 0.01,0.14 ± 0.01,Elastic Net
et,0.8 ± 0.01,0.97 ± 0.03,0.99 ± 0.01,0.47 ± 0.02,0.47 ± 0.01,0.49 ± 0.01,0.26 ± 0.03,Extra Trees Regressor
gbr,0.77 ± 0.01,0.88 ± 0.03,0.94 ± 0.02,0.52 ± 0.02,0.46 ± 0.01,0.45 ± 0.01,0.26 ± 0.02,Gradient Boosting Regressor
huber,0.73 ± 0.01,0.81 ± 0.02,0.9 ± 0.01,0.56 ± 0.01,0.44 ± 0.01,0.43 ± 0.0,0.15 ± 0.01,Huber Regressor
knn,0.8 ± 0.01,0.97 ± 0.02,0.98 ± 0.01,0.47 ± 0.01,0.47 ± 0.01,0.48 ± 0.0,0.16 ± 0.01,K Neighbors Regressor
lar,0.73 ± 0.01,0.8 ± 0.02,0.9 ± 0.01,0.56 ± 0.01,0.44 ± 0.0,0.43 ± 0.0,0.14 ± 0.01,Least Angle Regression


# Outros testes

In [ ]:
import pandas as pd
import 
substituicoes = {99:0,
                 'x':0,
                 'F':'f'
                 }

type_dict = {
    'data': 'datetime64[ns]',     # Datas
    'hora': 'datetime64[ns]',     # Horários
    'idade': 'Int64',             # Idades como inteiros (com suporte a NaN)
    'sexo': 'string',             # Sexo como string (categórico)
    'peso': 'Int64',            # Peso como número contínuo
    'altura': 'float64',          # Altura como número contínuo
    'vestimenta': 'string',       # Tipo de vestimenta (categórico)
    'p1': 'Int64',                 # Variáveis numéricas inteiras (suporte a NaN)
    'p2': 'Int64',
    'p3': 'Int64',
    'p4': 'Int64',
    'p5': 'Int64',
    'p6': 'Int64',
    'p7': 'Int64',
    'p8': 'Int64',
    'tev': 'float64',             # Dados contínuos
    'utci': 'float64',
    'sst': 'float64',
    'ste': 'float64',
    'psti': 'float64',
    'wbgt': 'float64',
    'wci': 'float64',
    'tek': 'float64',
    'te': 'float64',
    'pst': 'float64',
    'tmedia': 'float64',
    'tmax': 'float64',
    'tmin': 'float64',
    'tu': 'float64',
    'ur': 'float64',              # Umidade relativa
    'ur_max': 'float64',          # Umidade relativa máxima
    'ur_min': 'float64',          # Umidade relativa mínima
    'rsolarmed': 'float64',       # Radiação solar média
    'rsolartot': 'float64',       # Radiação solar total
    'vel_vento': 'float64',       # Velocidade do vento
    'dir_vento': 'float64',       # Direção do vento
    'sd_dirvento': 'float64',     # Desvio padrão da direção do vento
    'vel_vento_max': 'float64',   # Velocidade máxima do vento
    'dir_max_vento': 'float64',   # Direção máxima do vento
    'chuva_tot': 'float64'        # Total de chuva
}

df_bruto= df.copy()
from clearml.automation import PipelineDecorator
PipelineDecorator.run_locally()

df_processado = pipeline_processamento(df_bruto,
                       type_dict=type_dict,
                       substituicoes=substituicoes)



# MANIPUlÇÃO DE DATASETS CLEARML


In [ ]:
# UPLOAD
upload_dataset(df, dataset_name="dados_brutos", dataset_project="Datasets", description="Dataset bruto, sem processamento"
               , tags=["conforto_termico", "santa_maria", "brasil"])

## Listar datasets

In [ ]:
# LISTAR DATASETS
lista_datasets=Dataset.list_datasets()
pd.DataFrame(lista_datasets)
import clearml


In [ ]:
# DOWNLOAD
df= download_from_clearml('a155cb8c8a7f4d89b99bd05ce70cfd41', 'dados/baixados/')

In [ ]:
from pipelines.pipeline_treinamento import *
coluna_alvo="sensacao_termica"
atributos=["sensacao_termica","idade_anos","peso_kg","altura_cm","sexo_biologico",'temperatura_media_c','umidade_relativa_percent','radiacao_solar_media_wm2']

params = {
     "data": df_processado[atributos],
     "target": coluna_alvo,
     "session_id": 42,
     "normalize": True,
     "fold": 5,
     "verbose": False,
     "html": False,
     "use_gpu": True,
     "log_experiment": False,
 }
from clearml.automation import PipelineDecorator
PipelineDecorator.run_locally()
pipeline_treinamento(atributos,coluna_alvo, params)

#e,m, r = treinar_classificacao(params)

In [ ]:
# Importe as funções do seu script de pipeline
from pipeline_de_treinamento import step_treinar_modelo, step_avaliar_e_registrar_modelo

# Suponha que 'df_processado' é seu DataFrame pronto
# df_processado = ...
df_processado = df_processado.iloc[:300]
print("--- Testando a etapa de treino ---")
try:
    # Use .__wrapped__ para chamar a função interna diretamente
    artefatos = step_treinar_modelo.__wrapped__(df_treino=df_processado)
    print("Etapa de treino funcionou. Artefatos gerados:")
    print(artefatos)

    print("\n--- Testando a etapa de avaliação ---")
    # Agora teste a segunda etapa com os artefatos da primeira
    step_avaliar_e_registrar_modelo.__wrapped__(artefatos_treino=artefatos, df_treino=df_processado)
    print("Etapa de avaliação funcionou.")

except Exception as e:
    print("\n!!! Erro encontrado durante o teste local !!!")
    # Isso imprimirá o erro real imediatamente, sem o ClearML no meio.
    raise e

# Clearml-server

In [ ]:
from clearml.backend_api.session.client import APIClient
client = APIClient()
# pegue o projeto pelo nome (ou já use o ID, se souber)
#proj = client.projects.get_all(name='teste2')[0]
projetos = client.projects.get_all()
projetos = {projeto.name:projeto.id for projeto in projetos if projeto}
print(projetos)

In [ ]:
try:
    id=projetos['conforto_termddico']   
except:
    id=client.projects.create(name='conforto_termico').id
print(id)

In [ ]:
client.projects.delete(project=proj.id, force=True)  # falha se

In [ ]:
lista_datasets=Dataset.list_datasets()
pd.DataFrame(lista_datasets)

